## SVM
SVM本身是一个二类分类器，多类问题应用SVM需要对代码做一些修改

In [1]:
from numpy import *

def loadDataset(filename):
    dataMat = []
    labelMat = []
    fr = open(filename)
    for line in fr.readlines():
        lineArr = line.strip().split()
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

在某个区间范围内随机选择一个整数，只要函数值不等于输入值i，函数就会进行随机选择。
i是第一个alpha的下标，m是alpha的数目

In [2]:
def selectJrand(i,m):
    j = i
    while (j == i):
        j = int(random.uniform(0,m))
    return j

对于数值不在范围内时对其进行调整，调整大于H或小于L的alpha值

In [3]:
def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [4]:
dataArr,labelArr = loadDataset('testSet.txt')
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

### SMO算法
* 整个SMO算法包括两个部分：求解两个变量二次规划的解析方法和选择变量的启发式方法。
* 为了求解两个变量的二次规划问题，首先分析约束条件，然后在此约束条件下求极小
* SMO在每个子问题中选择两个变量优化，其中至少一个变量是违反KKT条件的
* 选择第1个变量的过程称外层循环。外层循环在训练样本中选取违反KKT条件最严重的样本点，并将其对应的变量作为第1个变量。该检验在容错率范围内进行。检验过程中，外层循环先遍历所有满足条件0<alpha_i<C的样本点，即在间隔边界上的支持向量点，检验它们是否满足KKT条件。
* 选择第2个变量的过程称为内层循环。假设外层循环中已经找到第1个变量alpha_1，第2个变量的选择标准是希望alpha_2有足够大的变化。为了加快计算速度，一种简单的方法是选择alpha_2使其对应的|E1-E2|最大，因为alpha_1已经确定，E1也确定，如果E1为正，则选择最小的Ei作为E2；如果E1为负，则选择最大的Ei作为E2。为了节省计算时间，将所有的Ei值保存在一个列表中。在特殊情况下，如果内层循环通过以上方法选择的alpha_2不能够使目标函数有足够的下降，那么采用启发式规则继续选择alpha_2：遍历在间隔边界上的支持向量点，依次将其对应的变量作为alpha_2试用，直到目标函数有足够的下降，若找不到合适的alpha_2，则遍历训练数据集；若仍找不到合适的alpha_2，则放弃第一个alpha_1，通过外层循环寻找别的alpha_1。
* 计算阈值b和差值Ei




#### 简化版SMO

In [ ]:
#简化版SMO伪码

#创建一个alpha向量并将其初始化为0向量
#当迭代次数小于最大迭代次数时（外循环）
    #对数据集中的每个数据向量（内循环）：
        #如果该数据向量可以被优化：
            #随机选择另外一个数据向量
            #同时优化这两个向量
            #如果两个向量都不能被优化，退出内循环
    #如果所有向量都没有被优化，增加迭代数目，继续下一次循环

In [5]:
#简化版SMO
#dataMat    ：数据列表
#classLabels：标签列表
#C          ：权衡因子（增加松弛因子而在目标优化函数中引入了惩罚项）
#toler      ：容错率
#maxIter    ：最大迭代次数
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()
    b = 0
    m,n = shape(dataMatrix)
    alphas = mat(zeros((m,1)))
    iter = 0
    while(iter < maxIter):
        alphaPairsChanged = 0
        for i in range (m):
            fXi = float(multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i,:].T)) + b
            Ei = fXi - float(labelMat[i])
            #如果alpha可以更改则进入优化过程
            if((labelMat[i] * Ei < -toler) and (alphas[i] < C) or (labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i, m) #随机选择第二个alpha
                fXj = float(multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                #保证alpha在C和0之间
                if(labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas [i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])

                if L == H:
                    print("L=H")
                    continue
                eta = 2.0 * dataMatrix[i,:] * dataMatrix[j,:].T - dataMatrix[i,:] * dataMatrix[i,:].T - dataMatrix[j,:] * dataMatrix[j,:].T
                if eta >= 0:
                    print("eta>=0")
                    continue
                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)
                if(abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough")
                    continue
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])  #对i进行修改，修改量与j相同但方向相反
                #设置常数项
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i,:] * dataMatrix[i,:].T - \
                              labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i,:] * dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[j, :].T - \
                              labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j, :] * dataMatrix[j, :].T
                if (0 < alphas[i]) and (C > alphas[j]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b1) / 2.0
                alphaPairsChanged += 1
                print("iter:%d  i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0):
            iter += 1
        else:
            iter = 0
        print("iteration number:%d" % iter)
    return b,alphas

In [6]:
b,alphas = smoSimple(dataArr,labelArr,0.6,0.001,40)

L=H
L=H
iter:0  i:2, pairs changed 1
L=H
iter:0  i:4, pairs changed 2
iter:0  i:5, pairs changed 3
L=H
L=H
L=H
iter:0  i:20, pairs changed 4
j not moving enough
L=H
j not moving enough
L=H
iter:0  i:29, pairs changed 5
L=H
L=H
L=H
L=H
j not moving enough
iter:0  i:39, pairs changed 6
L=H
j not moving enough
L=H
L=H
L=H
L=H
j not moving enough
iter:0  i:54, pairs changed 7
iter:0  i:55, pairs changed 8
iter:0  i:56, pairs changed 9
j not moving enough
j not moving enough
iter:0  i:69, pairs changed 10
iter:0  i:76, pairs changed 11
L=H
j not moving enough
L=H
j not moving enough
j not moving enough
j not moving enough
iteration number:0
iter:0  i:0, pairs changed 1
iter:0  i:2, pairs changed 2
j not moving enough
iter:0  i:5, pairs changed 3
L=H
j not moving enough
iter:0  i:18, pairs changed 4
j not moving enough
L=H
iter:0  i:26, pairs changed 5
j not moving enough
j not moving enough
iter:0  i:39, pairs changed 6
j not moving enough
iter:0  i:46, pairs changed 7
L=H
j not moving enou

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L=H
iteration number:1
j not moving enough
L=H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L=H
iteration number:2
j not moving enough
iter:2  i:23, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter:0  i:70, pairs changed 1
j not moving enough
j not moving enough
iteration number:0
iter:0  i:10, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j n

j not moving enough
iteration number:4
j not moving enough
j not moving enough
j not moving enough
iteration number:5
j not moving enough
j not moving enough
j not moving enough
iteration number:6
iter:6  i:17, pairs changed 1
j not moving enough
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j not moving enough
iteration number:1
j not moving enough
j not moving enough
j not moving enough
iteration number:2
j not moving enough
j not moving enough
j not moving enough
iteration number:3
j not moving enough
j not moving enough
j not moving enough
iteration number:4
j not moving enough
j not moving enough
j not moving enough
iteration number:5
j not moving enough
j not moving enough
j not moving enough
iteration number:6
j not moving enough
j not moving enough
j not moving enough
iteration number:7
j not moving enough
j not moving enough
j not moving enough
iteration number:8
j not moving enough
j not moving enough
j not moving enough
it

j not moving enough
j not moving enough
iteration number:13
j not moving enough
j not moving enough
iteration number:14
j not moving enough
j not moving enough
iteration number:15
j not moving enough
j not moving enough
iteration number:16
j not moving enough
j not moving enough
iteration number:17
j not moving enough
j not moving enough
iteration number:18
j not moving enough
j not moving enough
iteration number:19
j not moving enough
j not moving enough
iteration number:20
j not moving enough
j not moving enough
iteration number:21
j not moving enough
j not moving enough
iteration number:22
j not moving enough
j not moving enough
iteration number:23
j not moving enough
j not moving enough
iteration number:24
j not moving enough
j not moving enough
iteration number:25
j not moving enough
j not moving enough
iteration number:26
j not moving enough
j not moving enough
iteration number:27
j not moving enough
j not moving enough
iteration number:28
j not moving enough
j not moving enough


In [7]:
b

matrix([[-3.82507808]])

In [8]:
alphas[alphas > 0]

matrix([[0.16349813, 0.13473399, 0.07035245, 0.36858457]])

In [9]:
for i in range(100):
    if(alphas[i] > 0.0):
        print(dataArr[i], labelArr[i])

[4.658191, 3.507396] -1.0
[3.457096, -0.082216] -1.0
[2.893743, -1.643468] -1.0
[6.080573, 0.418886] 1.0


#### 完整版SMO
完整版应用了一些能够提速的启发方法：是通过一个外循环来选择第一个alpha值的，其选择过程会在两种方式之间进行交替，一种方式是在所有的数据集上进行单遍扫描，另一种方式则是在非边界alpha（非边界alpha指的是那些不等于边界0或C的alpha值）中实现单遍扫描。

In [10]:
#完整platt SMO算法的支持函数
class optStruct:
    def __init__(self,dataMatIn,classLabels,C,toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))  #缓存误差

#计算E值并返回，单独写出来方便调用
def calcEK(oS,k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T * (oS.X * oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

def selectJ(i,oS,Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1,Ei]
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if(len(validEcacheList))>1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEK(oS,k)
            deltaE = abs(Ei - Ek)
            if(deltaE > maxDeltaE): # 选择具有最大步长的j
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:
        j = selectJrand(i,oS.m)
        Ej = calcEK(oS,j)
    return j,Ej

def updateEk(oS,k):
    Ek = calcEK(oS,k)
    oS.eCache[k] = [1,Ek]


In [11]:
#完整platt SMO算法中的优化例程--寻找决策边界
def innerL(i,oS):
    Ei = calcEK(oS,i)
    if((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or \
      ((oS.labelMat[i] * Ei >  oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i,oS,Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if(oS.labelMat[i] != oS.labelMat[j]):
            L = max(0,oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L=H")
            return 0
        eta = 2.0 * oS.X[i,:] * oS.X[j,:].T - oS.X[i,:] * oS.X[i,:].T - oS.X[j,:] * oS.X[j,:].T
        if eta >=0:
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        if(abs(oS.alphas[j] - alphaJold) < 0.00001):
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS,i)
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i,:] * oS.X[i,:].T * \
                         oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[i,:] * oS.X[j,:].T
        b2 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i,:] * oS.X[j,:].T * \
                         oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[j,:] * oS.X[j,:].T
        if(0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif(0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

In [12]:
#完整platt SMO的外循环代码
def smoP(dataMatIn,classLabels,C,toler,maxIter,kTup=('lin',0)):
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while(iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
                print("fullSet,iter:%d, i:%d, pairs changed:%d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("non-bound,iter:%d, i:%d, pairs changed:%d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False
        elif(alphaPairsChanged == 0):
            entireSet = True
        print("iteration number:%d" % iter)
    return oS.b, oS.alphas

In [13]:
dataArr,labelArr = loadDataset('testSet.txt')
b,alphas = smoP(dataArr,labelArr,0.6,0.001,40)

fullSet,iter:0, i:0, pairs changed:1
j not moving enough
fullSet,iter:0, i:1, pairs changed:1
fullSet,iter:0, i:2, pairs changed:1
j not moving enough
fullSet,iter:0, i:3, pairs changed:1
fullSet,iter:0, i:4, pairs changed:1
fullSet,iter:0, i:5, pairs changed:1
fullSet,iter:0, i:6, pairs changed:1
j not moving enough
fullSet,iter:0, i:7, pairs changed:1
L=H
fullSet,iter:0, i:8, pairs changed:1
L=H
fullSet,iter:0, i:9, pairs changed:1
L=H
fullSet,iter:0, i:10, pairs changed:1
L=H
fullSet,iter:0, i:11, pairs changed:1
L=H
fullSet,iter:0, i:12, pairs changed:1
fullSet,iter:0, i:13, pairs changed:1
L=H
fullSet,iter:0, i:14, pairs changed:1
fullSet,iter:0, i:15, pairs changed:1
fullSet,iter:0, i:16, pairs changed:1
L=H
fullSet,iter:0, i:17, pairs changed:1
fullSet,iter:0, i:18, pairs changed:1
L=H
fullSet,iter:0, i:19, pairs changed:1
L=H
fullSet,iter:0, i:20, pairs changed:1
L=H
fullSet,iter:0, i:21, pairs changed:1
fullSet,iter:0, i:22, pairs changed:1
L=H
fullSet,iter:0, i:23, pairs chan

In [14]:
def calcWs(alphas, dataArr, classLabels):
    X = mat(dataArr)
    labelMat = mat(classLabels).transpose()
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i] * labelMat[i], X[i,:].T)
    return w

In [15]:
ws = calcWs(alphas,dataArr,labelArr)
print(ws)

[[ 0.38086802]
 [-0.12996948]]


### 在复杂数据上使用核函数
径向基函数是SVM中常用的一个核函数。径向基函数是一个采用向量作为自变量的函数，能够基于向量距离运算输出一个标量。这个距离可以使从<0,0>向量或者其他向量开始计算的距离。

In [16]:
#使用核函数
def kernelTrans(X, A, kTup):
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0] == 'lin':
        K = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow * deltaRow.T
        K = exp(K / (-1 * kTup[1] ** 2)) #元素间的除法
    else:
        raise NameError('we have a problem that kernel is not recognized')
    return K

In [17]:
#新版本
class optStructKernel:
    def __init__(self,dataMatIn,classLabels,C,toler,kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))  #缓存误差
        self.K = mat(zeros(self.m,self.m))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.C,self.X[i,:],kTup)

def innerLKernel(i,oS):
    Ei = calcEK(oS,i)
    if((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or \
      ((oS.labelMat[i] * Ei >  oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i,oS,Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if(oS.labelMat[i] != oS.labelMat[j]):
            L = max(0,oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L=H")
            return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,i]
        if eta >=0:
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej) / eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        if(abs(oS.alphas[j] - alphaJold) < 0.00001):
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS,i)
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i,i] - \
                         oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[i,j]
        b2 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.X[i,j] - \
                         oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.X[j,j]
        if(0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif(0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

def calcEKKernel(oS,k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T * oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

### 测试
代码只有一个可选的输入参数，该输入参数时高斯径向基函数中的一个用户定义变量核函数类型为'rbf

In [18]:
def testRbf(k1=1.3):
    dataArr,labelArr = loadDataset('testSetRBF.txt')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A>0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf',k1))
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]): errorCount += 1
    print("the training error rate is : %f" % (float(errorCount)/m))
    dataArr,labelArr = loadDataset('testSetRBF2.txt')
    errorCount = 0
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]): errorCount += 1
    print("the test error rate is: %f" % (float(errorCount)/m))

In [19]:
testRbf()

L=H
fullSet,iter:0, i:0, pairs changed:0
fullSet,iter:0, i:1, pairs changed:1
fullSet,iter:0, i:2, pairs changed:2
fullSet,iter:0, i:3, pairs changed:3
fullSet,iter:0, i:4, pairs changed:3
fullSet,iter:0, i:5, pairs changed:4
fullSet,iter:0, i:6, pairs changed:4
fullSet,iter:0, i:7, pairs changed:5
fullSet,iter:0, i:8, pairs changed:5
fullSet,iter:0, i:9, pairs changed:5
fullSet,iter:0, i:10, pairs changed:5
fullSet,iter:0, i:11, pairs changed:5
fullSet,iter:0, i:12, pairs changed:5
fullSet,iter:0, i:13, pairs changed:6
fullSet,iter:0, i:14, pairs changed:7
fullSet,iter:0, i:15, pairs changed:8
fullSet,iter:0, i:16, pairs changed:9
j not moving enough
fullSet,iter:0, i:17, pairs changed:9
fullSet,iter:0, i:18, pairs changed:9
fullSet,iter:0, i:19, pairs changed:9
j not moving enough
fullSet,iter:0, i:20, pairs changed:9
fullSet,iter:0, i:21, pairs changed:10
fullSet,iter:0, i:22, pairs changed:11
fullSet,iter:0, i:23, pairs changed:12
fullSet,iter:0, i:24, pairs changed:13
fullSet,iter

non-bound,iter:2, i:78, pairs changed:52
non-bound,iter:2, i:80, pairs changed:53
non-bound,iter:2, i:81, pairs changed:54
non-bound,iter:2, i:83, pairs changed:55
non-bound,iter:2, i:84, pairs changed:56
non-bound,iter:2, i:86, pairs changed:57
non-bound,iter:2, i:87, pairs changed:58
non-bound,iter:2, i:94, pairs changed:59
non-bound,iter:2, i:95, pairs changed:60
iteration number:3
non-bound,iter:3, i:0, pairs changed:1
non-bound,iter:3, i:1, pairs changed:2
non-bound,iter:3, i:2, pairs changed:3
non-bound,iter:3, i:3, pairs changed:4
non-bound,iter:3, i:5, pairs changed:5
non-bound,iter:3, i:7, pairs changed:6
non-bound,iter:3, i:13, pairs changed:7
non-bound,iter:3, i:14, pairs changed:8
non-bound,iter:3, i:15, pairs changed:9
non-bound,iter:3, i:16, pairs changed:10
non-bound,iter:3, i:20, pairs changed:11
non-bound,iter:3, i:21, pairs changed:12
non-bound,iter:3, i:22, pairs changed:13
non-bound,iter:3, i:23, pairs changed:14
non-bound,iter:3, i:24, pairs changed:15
non-bound,it

non-bound,iter:6, i:1, pairs changed:2
j not moving enough
non-bound,iter:6, i:2, pairs changed:2
j not moving enough
non-bound,iter:6, i:3, pairs changed:2
j not moving enough
non-bound,iter:6, i:7, pairs changed:2
j not moving enough
non-bound,iter:6, i:13, pairs changed:2
j not moving enough
non-bound,iter:6, i:14, pairs changed:2
j not moving enough
non-bound,iter:6, i:15, pairs changed:2
j not moving enough
non-bound,iter:6, i:16, pairs changed:2
non-bound,iter:6, i:20, pairs changed:3
non-bound,iter:6, i:21, pairs changed:4
non-bound,iter:6, i:22, pairs changed:5
j not moving enough
non-bound,iter:6, i:23, pairs changed:5
j not moving enough
non-bound,iter:6, i:24, pairs changed:5
j not moving enough
non-bound,iter:6, i:27, pairs changed:5
j not moving enough
non-bound,iter:6, i:28, pairs changed:5
j not moving enough
non-bound,iter:6, i:29, pairs changed:5
j not moving enough
non-bound,iter:6, i:33, pairs changed:5
non-bound,iter:6, i:38, pairs changed:6
non-bound,iter:6, i:40, 

j not moving enough
non-bound,iter:8, i:87, pairs changed:17
j not moving enough
non-bound,iter:8, i:94, pairs changed:17
j not moving enough
non-bound,iter:8, i:95, pairs changed:17
iteration number:9
non-bound,iter:9, i:0, pairs changed:1
j not moving enough
non-bound,iter:9, i:1, pairs changed:1
j not moving enough
non-bound,iter:9, i:2, pairs changed:1
j not moving enough
non-bound,iter:9, i:3, pairs changed:1
j not moving enough
non-bound,iter:9, i:7, pairs changed:1
j not moving enough
non-bound,iter:9, i:13, pairs changed:1
j not moving enough
non-bound,iter:9, i:14, pairs changed:1
non-bound,iter:9, i:15, pairs changed:2
j not moving enough
non-bound,iter:9, i:16, pairs changed:2
non-bound,iter:9, i:20, pairs changed:3
non-bound,iter:9, i:21, pairs changed:4
non-bound,iter:9, i:22, pairs changed:5
j not moving enough
non-bound,iter:9, i:23, pairs changed:5
j not moving enough
non-bound,iter:9, i:24, pairs changed:5
j not moving enough
non-bound,iter:9, i:27, pairs changed:5
j n

j not moving enough
non-bound,iter:11, i:63, pairs changed:14
j not moving enough
non-bound,iter:11, i:64, pairs changed:14
non-bound,iter:11, i:66, pairs changed:15
non-bound,iter:11, i:67, pairs changed:16
non-bound,iter:11, i:69, pairs changed:17
j not moving enough
non-bound,iter:11, i:71, pairs changed:17
j not moving enough
non-bound,iter:11, i:72, pairs changed:17
j not moving enough
non-bound,iter:11, i:74, pairs changed:17
j not moving enough
non-bound,iter:11, i:75, pairs changed:17
j not moving enough
non-bound,iter:11, i:76, pairs changed:17
j not moving enough
non-bound,iter:11, i:77, pairs changed:17
j not moving enough
non-bound,iter:11, i:78, pairs changed:17
j not moving enough
non-bound,iter:11, i:80, pairs changed:17
j not moving enough
non-bound,iter:11, i:81, pairs changed:17
j not moving enough
non-bound,iter:11, i:83, pairs changed:17
j not moving enough
non-bound,iter:11, i:84, pairs changed:17
j not moving enough
non-bound,iter:11, i:86, pairs changed:17
j not 

j not moving enough
non-bound,iter:14, i:28, pairs changed:0
j not moving enough
non-bound,iter:14, i:29, pairs changed:0
j not moving enough
non-bound,iter:14, i:33, pairs changed:0
j not moving enough
non-bound,iter:14, i:38, pairs changed:0
j not moving enough
non-bound,iter:14, i:40, pairs changed:0
j not moving enough
non-bound,iter:14, i:41, pairs changed:0
j not moving enough
non-bound,iter:14, i:43, pairs changed:0
j not moving enough
non-bound,iter:14, i:44, pairs changed:0
j not moving enough
non-bound,iter:14, i:48, pairs changed:0
j not moving enough
non-bound,iter:14, i:49, pairs changed:0
j not moving enough
non-bound,iter:14, i:50, pairs changed:0
j not moving enough
non-bound,iter:14, i:53, pairs changed:0
j not moving enough
non-bound,iter:14, i:54, pairs changed:0
j not moving enough
non-bound,iter:14, i:55, pairs changed:0
j not moving enough
non-bound,iter:14, i:56, pairs changed:0
j not moving enough
non-bound,iter:14, i:57, pairs changed:0
j not moving enough
non-